In [1]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install transformers
!pip install -U sentence-transformers
!pip install PyPDF2

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu
     |████████████████████████████████| 85 kB 960 kB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 5.8 MB/s eta 0:00:01
     |████████████████████████████████| 9.8 MB 3.0 MB/s eta 0:00:011
     |████████████████████████████████| 34.5 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 2.4 MB/s eta 0:00:01
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125922 sha256=7f89831b5df62f38609db3ee346ca0b0804e7771221aff4456a8296a9a7007bd
  Stored in directory: /home/riot/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [5]:
#!wget https://learn.microsoft.com/pdf?url=https%3A%2F%2Flearn.microsoft.com%2Fen-us%2Fazure%2Fsynapse-analytics%2Ftoc.json

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-03-08 20:04:30--  https://learn.microsoft.com/pdf?url=https%3A%2F%2Flearn.microsoft.com%2Fen-us%2Fazure%2Fsynapse-analytics%2Ftoc.json
Resolving learn.microsoft.com (learn.microsoft.com)... 95.100.65.213, 2a02:26f0:3400:18f::3544, 2a02:26f0:3400:182::3544
Connecting to learn.microsoft.com (learn.microsoft.com)|95.100.65.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 397216637 (379M) [application/pdf]
Saving to: ‘pdf?url=https:%2F%2Flearn.microsoft.com%2Fen-us%2Fazure%2Fsynapse-analytics%2Ftoc.json’

pdf?url=https:%2F%2 100%[===================>] 378.81M  10.6MB/s    in 39s     

2023-03-08 20:05:11 (9.67 MB/s) - ‘pdf?url=https:%2F

In [3]:
from PyPDF2 import PdfReader
from sentence_transformers import SentenceTransformer, util
from itertools import islice
import torch
import pickle

/home/riot/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
class PdfObject:
    def __init__(self, file_path):
        self.file_path = file_path
        self.reader = PdfReader(self.file_path)
        self.pages = len(self.reader.pages)
    
    
    def extract_pages_data(self, limit=None):
        limit = self.pages if not limit else limit
        extracted_data = []
        for i, page in enumerate(islice(self.reader.pages, limit)):
            
            extracted_data.append(
                {
                    "page": i, 
                    "text": page.extract_text()
                }
            )
            print(f"{i}-th page extracted")
        return extracted_data


class Embeddings:
    def __init__(self, model, corpus=None, corpus_file_path=None, embeddings_file_path=None):
        self.model = model
        
        if corpus:
            self.corpus = corpus
        elif corpus_file_path and embeddings_file_path:
            self.load_from_files(corpus_file_path, embeddings_file_path)
            
    
    def _stack(self):
        self.embeddings = torch.stack(self.embeddings)
        
    def create(self):
        corpus_emds = []
        corpus_size = len(self.corpus)
        percents = set()
        for i, page in enumerate(corpus):
            page_text = page["text"]
            embds = self.model.encode(page_text, convert_to_tensor=True)

            percent_processed = int((i/corpus_size)*100)
            if percent_processed and percent_processed % 10 == 0:
                if percent_processed not in percents:
                    print(f"{percent_processed}% processed")
                    percents.add(percent_processed)

            corpus_emds.append(embds)
        
        self.embeddings = corpus_emds
        self._stack()
        return self.embeddings
    
    def save_to_files(self, corpus_store_path, embeddings_store_path):
        torch.save(self.embeddings, embeddings_store_path)
        
        with open(corpus_store_path, 'wb') as handle:
            pickle.dump(self.corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    def load_from_files(self, corpus_store_path, embeddings_store_path):
        with open(corpus_store_path, 'rb') as handle:
            self.corpus = pickle.load(handle)
        
        self.embeddings = torch.load(embeddings_store_path) 

        
class Prediction:
    def __init__(self, model, corpus, embeddings):
        self.corpus = corpus
        self.model = model
        self.embeddings = embeddings
    
    def predict(self, query, limit):
        top_k = limit
        query_embedding = self.model.encode(query, convert_to_tensor=True)
        # We use cosine-similarity and torch.topk to find the highest 5 scores
        cos_scores = util.cos_sim(query_embedding, self.embeddings)[0]

        top_results = torch.topk(cos_scores, k=top_k)

        print("\n\n======================\n\n")
        print("Query:", query)
        print(f"\nTop {top_k} most similar sentences in corpus:")

        for score, idx in zip(top_results[0], top_results[1]):
            print(f"\n{self.corpus[idx]}\n", "(Score: {:.4f})".format(score))

In [82]:
# entrypoint
synapse_book = PdfObject('azure_synapse_analytics.pdf')
corpus = synapse_book.extract_pages_data()

0-th page extracted
1-th page extracted
2-th page extracted
3-th page extracted
4-th page extracted
5-th page extracted
6-th page extracted
7-th page extracted
8-th page extracted
9-th page extracted
10-th page extracted
11-th page extracted
12-th page extracted
13-th page extracted
14-th page extracted
15-th page extracted
16-th page extracted
17-th page extracted
18-th page extracted
19-th page extracted
20-th page extracted
21-th page extracted
22-th page extracted
23-th page extracted
24-th page extracted
25-th page extracted
26-th page extracted
27-th page extracted
28-th page extracted
29-th page extracted
30-th page extracted
31-th page extracted
32-th page extracted
33-th page extracted
34-th page extracted
35-th page extracted
36-th page extracted
37-th page extracted
38-th page extracted
39-th page extracted
40-th page extracted
41-th page extracted
42-th page extracted
43-th page extracted
44-th page extracted
45-th page extracted
46-th page extracted
47-th page extracted
48

385-th page extracted
386-th page extracted
387-th page extracted
388-th page extracted
389-th page extracted
390-th page extracted
391-th page extracted
392-th page extracted
393-th page extracted
394-th page extracted
395-th page extracted
396-th page extracted
397-th page extracted
398-th page extracted
399-th page extracted
400-th page extracted
401-th page extracted
402-th page extracted
403-th page extracted
404-th page extracted
405-th page extracted
406-th page extracted
407-th page extracted
408-th page extracted
409-th page extracted
410-th page extracted
411-th page extracted
412-th page extracted
413-th page extracted
414-th page extracted
415-th page extracted
416-th page extracted
417-th page extracted
418-th page extracted
419-th page extracted
420-th page extracted
421-th page extracted
422-th page extracted
423-th page extracted
424-th page extracted
425-th page extracted
426-th page extracted
427-th page extracted
428-th page extracted
429-th page extracted
430-th pag

764-th page extracted
765-th page extracted
766-th page extracted
767-th page extracted
768-th page extracted
769-th page extracted
770-th page extracted
771-th page extracted
772-th page extracted
773-th page extracted
774-th page extracted
775-th page extracted
776-th page extracted
777-th page extracted
778-th page extracted
779-th page extracted
780-th page extracted
781-th page extracted
782-th page extracted
783-th page extracted
784-th page extracted
785-th page extracted
786-th page extracted
787-th page extracted
788-th page extracted
789-th page extracted
790-th page extracted
791-th page extracted
792-th page extracted
793-th page extracted
794-th page extracted
795-th page extracted
796-th page extracted
797-th page extracted
798-th page extracted
799-th page extracted
800-th page extracted
801-th page extracted
802-th page extracted
803-th page extracted
804-th page extracted
805-th page extracted
806-th page extracted
807-th page extracted
808-th page extracted
809-th pag

1154-th page extracted
1155-th page extracted
1156-th page extracted
1157-th page extracted
1158-th page extracted
1159-th page extracted
1160-th page extracted
1161-th page extracted
1162-th page extracted
1163-th page extracted
1164-th page extracted
1165-th page extracted
1166-th page extracted
1167-th page extracted
1168-th page extracted
1169-th page extracted
1170-th page extracted
1171-th page extracted
1172-th page extracted
1173-th page extracted
1174-th page extracted
1175-th page extracted
1176-th page extracted
1177-th page extracted
1178-th page extracted
1179-th page extracted
1180-th page extracted
1181-th page extracted
1182-th page extracted
1183-th page extracted
1184-th page extracted
1185-th page extracted
1186-th page extracted
1187-th page extracted
1188-th page extracted
1189-th page extracted
1190-th page extracted
1191-th page extracted
1192-th page extracted
1193-th page extracted
1194-th page extracted
1195-th page extracted
1196-th page extracted
1197-th pag

1525-th page extracted
1526-th page extracted
1527-th page extracted
1528-th page extracted
1529-th page extracted
1530-th page extracted
1531-th page extracted
1532-th page extracted
1533-th page extracted
1534-th page extracted
1535-th page extracted
1536-th page extracted
1537-th page extracted
1538-th page extracted
1539-th page extracted
1540-th page extracted
1541-th page extracted
1542-th page extracted
1543-th page extracted
1544-th page extracted
1545-th page extracted
1546-th page extracted
1547-th page extracted
1548-th page extracted
1549-th page extracted
1550-th page extracted
1551-th page extracted
1552-th page extracted
1553-th page extracted
1554-th page extracted
1555-th page extracted
1556-th page extracted
1557-th page extracted
1558-th page extracted
1559-th page extracted
1560-th page extracted
1561-th page extracted
1562-th page extracted
1563-th page extracted
1564-th page extracted
1565-th page extracted
1566-th page extracted
1567-th page extracted
1568-th pag

...

6239-th page extracted
6240-th page extracted
6241-th page extracted
6242-th page extracted
6243-th page extracted
6244-th page extracted
6245-th page extracted
6246-th page extracted
6247-th page extracted
6248-th page extracted
6249-th page extracted
6250-th page extracted
6251-th page extracted
6252-th page extracted
6253-th page extracted
6254-th page extracted
6255-th page extracted
6256-th page extracted
6257-th page extracted
6258-th page extracted
6259-th page extracted
6260-th page extracted
6261-th page extracted
6262-th page extracted
6263-th page extracted
6264-th page extracted
6265-th page extracted
6266-th page extracted
6267-th page extracted
6268-th page extracted
6269-th page extracted
6270-th page extracted
6271-th page extracted
6272-th page extracted
6273-th page extracted
6274-th page extracted
6275-th page extracted
6276-th page extracted
6277-th page extracted
6278-th page extracted
6279-th page extracted
6280-th page extracted
6281-th page extracted
6282-th pag

6603-th page extracted
6604-th page extracted
6605-th page extracted
6606-th page extracted
6607-th page extracted
6608-th page extracted
6609-th page extracted
6610-th page extracted
6611-th page extracted
6612-th page extracted
6613-th page extracted
6614-th page extracted
6615-th page extracted
6616-th page extracted
6617-th page extracted
6618-th page extracted
6619-th page extracted
6620-th page extracted
6621-th page extracted
6622-th page extracted
6623-th page extracted
6624-th page extracted
6625-th page extracted
6626-th page extracted
6627-th page extracted
6628-th page extracted
6629-th page extracted
6630-th page extracted
6631-th page extracted
6632-th page extracted
6633-th page extracted
6634-th page extracted
6635-th page extracted
6636-th page extracted
6637-th page extracted
6638-th page extracted
6639-th page extracted
6640-th page extracted
6641-th page extracted
6642-th page extracted
6643-th page extracted
6644-th page extracted
6645-th page extracted
6646-th pag

6966-th page extracted
6967-th page extracted
6968-th page extracted
6969-th page extracted
6970-th page extracted
6971-th page extracted
6972-th page extracted
6973-th page extracted
6974-th page extracted
6975-th page extracted
6976-th page extracted
6977-th page extracted
6978-th page extracted
6979-th page extracted
6980-th page extracted
6981-th page extracted
6982-th page extracted
6983-th page extracted
6984-th page extracted
6985-th page extracted
6986-th page extracted
6987-th page extracted
6988-th page extracted
6989-th page extracted
6990-th page extracted
6991-th page extracted
6992-th page extracted
6993-th page extracted
6994-th page extracted
6995-th page extracted
6996-th page extracted
6997-th page extracted
6998-th page extracted
6999-th page extracted
7000-th page extracted
7001-th page extracted
7002-th page extracted
7003-th page extracted
7004-th page extracted
7005-th page extracted
7006-th page extracted
7007-th page extracted
7008-th page extracted
7009-th pag

7325-th page extracted
7326-th page extracted
7327-th page extracted
7328-th page extracted
7329-th page extracted
7330-th page extracted
7331-th page extracted
7332-th page extracted
7333-th page extracted
7334-th page extracted
7335-th page extracted
7336-th page extracted
7337-th page extracted
7338-th page extracted
7339-th page extracted
7340-th page extracted
7341-th page extracted
7342-th page extracted
7343-th page extracted
7344-th page extracted
7345-th page extracted
7346-th page extracted
7347-th page extracted
7348-th page extracted
7349-th page extracted
7350-th page extracted
7351-th page extracted
7352-th page extracted
7353-th page extracted
7354-th page extracted
7355-th page extracted
7356-th page extracted
7357-th page extracted
7358-th page extracted
7359-th page extracted
7360-th page extracted
7361-th page extracted
7362-th page extracted
7363-th page extracted
7364-th page extracted
7365-th page extracted
7366-th page extracted
7367-th page extracted
7368-th pag

7692-th page extracted
7693-th page extracted
7694-th page extracted
7695-th page extracted
7696-th page extracted
7697-th page extracted
7698-th page extracted
7699-th page extracted
7700-th page extracted
7701-th page extracted
7702-th page extracted
7703-th page extracted
7704-th page extracted
7705-th page extracted
7706-th page extracted
7707-th page extracted
7708-th page extracted
7709-th page extracted
7710-th page extracted
7711-th page extracted
7712-th page extracted
7713-th page extracted
7714-th page extracted
7715-th page extracted
7716-th page extracted
7717-th page extracted
7718-th page extracted
7719-th page extracted
7720-th page extracted
7721-th page extracted
7722-th page extracted
7723-th page extracted
7724-th page extracted
7725-th page extracted
7726-th page extracted
7727-th page extracted
7728-th page extracted
7729-th page extracted
7730-th page extracted
7731-th page extracted
7732-th page extracted
7733-th page extracted
7734-th page extracted
7735-th pag

In [5]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = Embeddings(model, corpus_file_path='data/corpus.pickle', embeddings_file_path='data/embeddings.pt')
#embeddings.create()
#embeddings.save_to_files('corpus.pickle', 'embeddings.pt')

In [6]:
chat = Prediction(model, embeddings.corpus, embeddings.embeddings)

query = input("Type your query: ")
count_results = int(input("How much results: "))
chat.predict(query, count_results)

Type your query: View
How much results: 1




Query: View

Top 1 most similar sentences in corpus:

{'page': 691, 'text': 'Example for SQL Login - T ype your User name and password.\n5. Select the view usPopulationView, and then select Load.\n'}
 (Score: 0.4219)


In [7]:
# add pinecone
!pip install pinecone-client
!pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [34]:
import pinecone
import os

%reload_ext dotenv
%dotenv

PINECONE_API_TOKEN = os.environ.get("PINECONE_API_TOKEN")
PINECONE_ENVIRONMENT = 'us-west1-gcp'
PINECONE_INDEX = 'super-brain'

pinecone.init(api_key=PINECONE_API_TOKEN, environment=PINECONE_ENVIRONMENT)

In [113]:
import uuid
import itertools

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = list(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = list(itertools.islice(it, batch_size))

def prepare_embeddings(embeddings):
    return ((str(uuid.uuid4()), embedding.tolist(), {'file':'azure_synapse_analytics_documentation', 'page': page}) for page, embedding in enumerate(embeddings))
        

def upsert_to_pinecone(index, embeddings, pool_threads=30, batch_size=100):
    with pinecone.Index(index, pool_threads=pool_threads) as index:
        # Send requests in parallel
        async_results = [
            index.upsert(ids_vectors_chunk, async_req=True)
            for ids_vectors_chunk in chunks(prepare_embeddings(embeddings), batch_size=batch_size)
        ]
        # Wait for and retrieve responses (this raises in case of error)
        responses = [async_result.get() for async_result in async_results]
        print(responses)


def query_pinecone(index_name, query, model, embeddings, corpus, limit=5):
    index = pinecone.Index(index_name)
    query_embedding = model.encode(query).tolist()
    query_response = index.query(
        vector=query_embedding,
        top_k=limit,
        include_values=True,
        include_metadata=True
    )['matches']
    results = []
    for match in query_response:
        values = match["values"]
        for ind, embedding in enumerate(embeddings):
            embedding = [round(emb, 10) for emb in embedding.tolist()]
            if embedding[0] == values[0]:
                results.append(
                    {
                        "match": match["score"],
                        "text": corpus[ind]
                    }
                )
    return results

In [114]:
#upsert_to_pinecone(PINECONE_INDEX, embeddings.embeddings)

In [117]:
import pprint

results = query_pinecone(PINECONE_INDEX, 'Create External View', model, embeddings.embeddings, embeddings.corpus)
pprint.pprint(results)

[{'match': 0.419137,
  'text': {'page': 691,
           'text': 'Example for SQL Login - T ype your User name and '
                   'password.\n'
                   '5. Select the view usPopulationView, and then select '
                   'Load.\n'}},
 {'match': 0.393314809,
  'text': {'page': 685,
           'text': 'the following script you can see how to add a new user '
                   'that will be authenticated using\n'
                   'Azure AD identity:\n'
                   'SQL\n'
                   'Instead of Azure AD principals, you can create SQL '
                   'principals that authenticate with the\n'
                   'login name and password.\n'
                   'SQL\n'
                   'In both cases, you can assign permissions to the users.\n'
                   'SQL\n'
                   'The security rules depend on your security policies. Some '
                   'generic guidelines are:\n'
                   'You should deny ADMINISTER DATAB